In [1]:
#Importing the dataset
import pandas as pd

df1=pd.read_csv('textdata.csv').fillna(' ')

In [2]:
df2=pd.read_csv('texttestdata.csv').fillna(' ')

In [3]:
df=pd.concat([df1,df2])

In [4]:
df1.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [5]:
df.shape

(312735, 8)

In [6]:
#Removing special characters from comment_text
import re

def clean_text(text):
    return re.sub(r'\W+',' ', text)

In [7]:
df['comment_text_clean']=df['comment_text']
df['comment_text_clean']=df['comment_text_clean'].apply(clean_text)

In [8]:
#Train data
df1['comment_text_clean']=df1['comment_text']
df1['comment_text_clean']=df1['comment_text_clean'].apply(clean_text)
#Test data
df2['comment_text_clean']=df2['comment_text']
df2['comment_text_clean']=df2['comment_text_clean'].apply(clean_text)

In [9]:
#FormattingText
def changecase(text):
    return text.lower()

In [10]:
df['comment_text_clean']=df['comment_text']
df['comment_text_clean']=df['comment_text_clean'].apply(changecase)

In [11]:
#Train data
df1['comment_text_clean']=df1['comment_text']
df1['comment_text_clean']=df1['comment_text_clean'].apply(changecase)
#Test data
df2['comment_text_clean']=df2['comment_text']
df2['comment_text_clean']=df2['comment_text_clean'].apply(changecase)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer=CountVectorizer(stop_words='english', min_df=0.0001,analyzer='word',max_features=10000,ngram_range=(1,2))
#X_train=vectorizer.fit(df['comment_text_clean'])
X_train=vectorizer.fit_transform(df1['comment_text_clean'])
#X_train=vectorizer.transform(df1['comment_text_clean'])
X_test=vectorizer.transform(df2['comment_text_clean'])

In [13]:
#Check if the dataset is balanced -
len(df[df['identity_hate']==1])

1405

In [14]:
df.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
target = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
y_train=df1[target]
y_train

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
159566,0,0,0,0,0,0
159567,0,0,0,0,0,0
159568,0,0,0,0,0,0
159569,0,0,0,0,0,0


In [16]:
#Building SVC model
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split

X_train, X_calib, y_train, y_calib=train_test_split(X_train,y_train,test_size=0.2)

svm_clf_m1=LinearSVC(random_state=42)
cali_clf_m1=CalibratedClassifierCV(base_estimator=svm_clf_m1,cv=5)
result=pd.DataFrame()

for labels in target:
    svm_clf_m1.fit(X_train,y_train[labels])
    cali_clf_m1.fit(X_calib,y_calib[labels])
    result[labels]=svm_clf_m1.predict(X_train)

result

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,1,1,1,1,1,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
127651,0,0,0,0,0,0
127652,0,0,0,0,0,0
127653,0,0,0,0,0,0
127654,0,0,0,0,0,0


In [17]:
len(result[result['toxic']==1])

10434

In [18]:
#Using TFIDF as the feature 
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english',min_df=0.0001,analyzer='word',max_features=10000,ngram_range=(1,2))
#X_train_2=tfidf_vectorizer.fit(df['comment_text_clean'])
X_train_2=tfidf_vectorizer.fit_transform(df1['comment_text_clean'])
#X_train_2=tfidf_vectorizer.transform(df1['comment_text_clean'])
X_test_2=tfidf_vectorizer.transform(df2['comment_text_clean'])

In [19]:
X_train_2.shape

(159571, 10000)

In [20]:
y_train_2 = df1[target]
X_train_2, X_calib_2, y_train_2, y_calib_2=train_test_split(X_train_2,y_train_2,test_size=0.2)

In [21]:
result_1=pd.DataFrame()
svm_clf_m2=LinearSVC(random_state=42)
cali_clf_m2=CalibratedClassifierCV(base_estimator=svm_clf_m2,cv=5)

for labels in target:
    svm_clf_m2.fit(X_train_2,y_train_2[labels])
    cali_clf_m2.fit(X_calib_2,y_calib_2[labels])
    result_1[labels]=svm_clf_m2.predict(X_train_2)

result_1

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
127651,0,0,0,0,0,0
127652,0,0,0,0,0,0
127653,0,0,0,0,0,0
127654,0,0,0,0,0,0


In [22]:
len(result_1[result_1['toxic']==1])

9801

In [1]:
from scipy.sparse import csr_matrix
X_test_2=csr_matrix.toarray(X_test_2)

NameError: name 'X_test_2' is not defined

In [ ]:
#Running predict function on both models
import numpy as np
result_test_m1=pd.DataFrame()
result_test_m2=pd.DataFrame()
test_val=0
X_test=np.array(X_test)

for labels in target:
    result_test_m1[labels]=cali_clf_m1.predict_proba(X_test)[:5,:]
#result_test_m2[labels]=cali_clf_m2.predict_proba(X_test)[:5,:]

#print(len(result_test_m1[result_test_m1['toxic']==1]),len(result_test_m2[result_test_m2['toxic']==1]))

In [ ]:
result_test_m2['toxic'].value_counts()

In [ ]:
#Creating Logistic Regression model
from sklearn.linear_model import LogisticRegression
log_clf=LogisticRegression()
result_log=pd.DataFrame()

for labels in target:
    log_clf.fit(X_train,df1[labels])
    result_log[labels]=log_clf.predict(X_train)
    
result_log

In [ ]:
result_log['toxic'].value_counts()

In [ ]:
#RandomForest model
from sklearn.ensemble import RandomForestClassifier
forest_clf=RandomForestClassifier(n_estimators=50,max_depth=200,random_state=42)
result_forest=pd.DataFrame()

for labels in target:
    forest_clf.fit(X_train,df1[labels])
    result_forest[labels]=forest_clf.predict(X_train)
    
result_forest

In [ ]:
result_forest['toxic'].value_counts()

In [ ]:
#Evaluating Logistic and RandomForest model on Test data
result_test_log=pd.DataFrame()
result_test_forest=pd.DataFrame()

for labels in target:
    result_test_log[labels]=log_clf.predict(X_test)
    result_test_forest[labels]=forest_clf.predict(X_test)
    
print(len(result_test_log[result_test_log['toxic']==1]),len(result_test_forest[result_test_forest['toxic']==1]))